# **<center><font style="color:rgb(100,109,254)">Module 1: Creating a Live Gesture Control Application</font> </center>**

<img src='https://drive.google.com/uc?export=download&id=10xY6U-4CecBuMkbONZp6bPx7BAZB6xfX'>

## **<font style="color:rgb(134,19,348)"> Module Outline </font>**

The module can be split into the following parts:

- *Lesson 1: Introduction to hand landmark detection theory*

- *Lesson 2:* Create a Hands Landmarks Detector

- *Lesson 3:* Build a Hands Fingers Counter

- *Lesson 4:* Build a Hand Gesture Recognizer 

- *Lesson 5:* Distance measurement

- *Lesson 6:* Brightness, Saturation, Contrast Enhancement & Gamma Correction

- ***Lesson 7:* Build the Final Application** *(This Tutorial)*

**Please Note**, these Jupyter Notebooks are not for sharing; do read the Copyright message below the Code License Agreement section which is in the last cell of this notebook.
-Taha Anwar

Alright, let's get started.

### **<font style="color:rgb(134,19,348)"> Import the Libraries</font>**

First we will import the required libraries.

In [1]:
import cv2
import mediapipe as mp
from collections import deque
from scipy.interpolate import interp1d
from previous_lesson import (detectHandsLandmarks, recognizeGestures,
                             calculateDistance, changeSatValue,
                             changeContrast, gammaCorrection)

## **<font style="color:rgb(134,19,348)">Initialize the Hands Landmarks Detection Model</font>**

After that, we will need to initialize the **`mp.solutions.hands`** class and then set up the **`mp.solutions.hands.Hands()`** function with appropriate arguments and also initialize **`mp.solutions.drawing_utils`** class that is required to visualize the detected landmarks, as we have been doing in the previous lessons.

In [2]:
# Initialize the mediapipe hands class.
mp_hands = mp.solutions.hands

# Set up the Hands functions for images and videos.
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.3)
hands_videos = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.8)

# Initialize the mediapipe drawing class.
mp_drawing = mp.solutions.drawing_utils

## **<font style="color:rgb(134,19,348)">Build the Final Application</font>**

Now finally, we will put all the functions we created in the previous lessons, together to build the final application. We will change the Brightness, Saturation, Contrast, and Gamma Correction of our webcam feed in real-time utilizing our hand gestures.

In [4]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0, cv2.CAP_DSHOW)


# Create named window for resizing purposes.
cv2.namedWindow('Live Gesture Control Application', cv2.WINDOW_NORMAL)

# Initialize the minimum brightness and saturation scale factors.
min_brightness = min_saturation = 0

# Initialize the maximum brightness and saturation scale factors.
max_brightness = max_saturation = 3

# Initialize the minimum and maximum contrast scale factors. 
min_contrast=1
max_contrast=3

# Initialize the minimum and maximum gamma scale factors.
min_gamma=0.1
max_gamma=3

# Initialize a buffer to store recognized gestures.
buffer = deque([], maxlen=20)

# Initialize some variables to store the change state.
change_brightness = change_saturation = change_contrast = change_gamma = False

# Initialize a variable to store the current mode.
current_mode = ''

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
   
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly then continue to the next iteration to read the next frame.
    if not ok:
        continue
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)
    
    # Get the height and width of the frame of the webcam video.
    frame_height, frame_width, _ = frame.shape
    
    # Perform Hands landmarks detection on the frame.
    _, results = detectHandsLandmarks(frame, hands_videos, draw=True, display=False)
    
    # Check if the hands landmarks in the frame are detected.
    if results.multi_hand_landmarks:
        
        # Perform left hand gesture recognition.
        current_gesture, size_gesture_tip_pts = recognizeGestures(frame, results,
                                                                  hand_label='LEFT',
                                                                  draw=False, display=False)
        
        # Check if a known gesture is recognized.
        if current_gesture != 'UNKNOWN':
            
            # Check if all the gestures stored in the buffer are equal to the current gesture.
            if all(current_gesture==gesture for gesture in buffer):
                
                # Append the current gesture into the buffer.
                buffer.append(current_gesture)
                
            # Otherwise.
            else:
                
                # Clear the buffer.
                buffer.clear()
            
            # Check if the length of the buffer is equal to 20.
            if len(buffer) == 20:
                
                # Calculate the distance between the middle finger tip and thumb tip landmark of the right hand.
                distance = calculateDistance(frame, size_gesture_tip_pts['MIDDLE'],
                                             size_gesture_tip_pts['THUMB'], display=False)
                
                # Check if the distance is calculated successfully.
                # This will be none in case when the hand is not in the frame.
                if distance:
                    
                    # Check if the current hand gesture is INDEX POINTING UP.
                    if current_gesture == 'INDEX POINTING UP':
                        
                        # Get the interpolation function.
                        brightness_interp_f = interp1d([30,230], [min_brightness, max_brightness])
                        
                        # Calculate the brighness scale factor based on the calculated distance.
                        # Higher the distance, higher the brighness scale factor will be.
                        brightness_scale = brightness_interp_f(distance)
                        
                        # Update the current mode and change state.
                        current_mode = 'BRIGHTNESS'
                        change_brightness = True
                    
                    # Check if the current hand gesture is VICTORY.
                    elif current_gesture == 'VICTORY':
                        
                        # Get the interpolation function and calculate the saturation scale factor.
                        saturation_interp_f = interp1d([30,230], [min_saturation, max_saturation])
                        saturation_scale = saturation_interp_f(distance)
                        
                        # Update the current mode and change state.
                        current_mode = 'SATURATION'
                        change_saturation = True
                    
                    # Check if the current hand gesture is SPIDERMAN.
                    elif current_gesture == 'PINKY':
                        
                        # Get the interpolation function and calculate the contrast scale factor.
                        contrast_interp_f = interp1d([30,230], [min_contrast, max_contrast])
                        contrast_scale = contrast_interp_f(distance)
                        
                        # Update the current mode and change state.
                        current_mode = 'CONTRAST'
                        change_contrast = True
                    
                    # Check if the current hand gesture is HIGH-FIVE.
                    elif current_gesture == 'HIGH-FIVE':
                        
                        # Get the interpolation function and calculate the gamma scale factor.
                        gamma_interp_f = interp1d([30,230], [min_gamma, max_gamma])
                        gamma_scale = gamma_interp_f(distance)
                        
                        # Update the current mode and change state.
                        current_mode = 'GAMMA CORRECTION'
                        change_gamma = True
                    
                    # Get the interpolation function and calculate the bar value.
                    # This will be used to draw a filled rectangle of height varying with the distance.
                    bar_interp_f = interp1d([30,230],  [frame_height-50, frame_height-400])
                    bar_value = bar_interp_f(distance)  
                    
                    # Write the current mode on the frame.
                    cv2.putText(frame, f'{current_mode} {(distance-30)//2}%',
                                (frame_width//2-(20*len(current_mode)), 40),
                                cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)
                    
                    # Draw the filled rectangle with varying height on the frame.
                    cv2.rectangle(frame, (frame_width-80, int(bar_value)), 
                                  (frame_width-50, frame_height-50), (255, 0, 255), -1)
                    
                    # Draw another rectangle around the filled rectangle on the frame.
                    cv2.rectangle(frame, (frame_width-80, frame_height-400),
                                  (frame_width-50, frame_height-50), (0, 255, 0), 6)
    # Otherwise.
    else:
        # Clear the buffer.
        buffer.clear()
        
    # Check if the change brighness state is true.
    if change_brightness:
        
        # Change the brighness of the frame. 
        frame = changeSatValue(frame, scale_factor=brightness_scale,
                               channel='Value', display=False)
    
    # Check if the change saturation state is true.
    if change_saturation:
        
        # Change the saturation of the frame. 
        frame = changeSatValue(frame, scale_factor=saturation_scale,
                               channel='Saturation', display=False) 
    
    # Check if the change contrast state is true.
    if change_contrast:
        
        # Change the contrast of the frame. 
        frame = changeContrast(frame, contrast_scale, display=False)
    
    # Check if the change gamma state is true.
    if change_gamma:
        
        # Perform gamma correction on the frame. 
        frame = gammaCorrection(frame, gamma_scale, display=False)
    
    # Display the frame.
    cv2.imshow('Live Gesture Control Application', frame)
    
    # Wait for 1ms. If a key is pressed, retreive the ASCII code of the key.
    k = cv2.waitKey(1) & 0xFF
    
    # Check if 'ESC' is pressed and break the loop.
    if(k == 27):
        break

# Release the VideoCapture Object and close the windows.
camera_video.release()
cv2.destroyAllWindows()




# Additional comments:
#       - In summary, this program counts the number of fingers
#         that are up. Then, it checks which ones are up to 
#         determine what the gesture is.

As expected, the final results are incredible!

### **<font style="color:rgb(255,140,0)"> Code License Agreement </font>**
```
Copyright (c) 2022 Bleedai.com

Feel free to use this code for your own projects commercial or noncommercial, these projects can be Research-based, just for fun, for-profit, or even Education with the exception that you’re not going to use it for developing a course, book, guide, or any other educational products.

Under *NO CONDITION OR CIRCUMSTANCE* you may use this code for your own paid educational or self-promotional ventures without written consent from Taha Anwar (BleedAI.com).

```